In [ ]:
!pip install langchain
!pip install jq
!pip install sentence-transformers

In [8]:
from huggingface_hub import hf_hub_download, snapshot_download

In [ ]:
snapshot_download(repo_id="JetBrains-Research/commit-chronicle", repo_type="dataset", cache_dir='./')

In [11]:
import pyarrow.parquet as pq

folder_path = './datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/data'  # 文件夹的路径

df = pq.read_table(folder_path + '/train-00000-of-00061-2a7ccc8e843f5f5b.parquet').to_pandas()

In [17]:
len(df.iloc[0]['mods'][0]['diff'])

977

In [31]:
import pyarrow.parquet as pq
import glob
import os
from tqdm.auto import tqdm

# Create empty DataFrames for each language
languages = ['Java', 'Python', 'C#', 'C++', 'JavaScript']
dfs = {lang: [] for lang in languages}

# Create a tqdm progress bar for each language
bars = {lang: tqdm(total=100000, desc=lang) for lang in languages}

folder_path = './datasets--JetBrains-Research--commit-chronicle/snapshots/5fd076e67b812a9f3d1999e5e40f71715f84bb51/data'  # 文件夹的路径

files = glob.glob(os.path.join(folder_path, 'train*'))
for file in files:
    df = pq.read_table(file).to_pandas()

    # Iterate over each language
    for lang in languages:
        # Filter rows where language column matches the current language
        lang_df = df[(df['language'] == lang) & (df['mods'].apply(len) == 1) & (df['mods'].apply(lambda x: x[0]['change_type']) == 'MODIFY') & (df['mods'].apply(lambda x: len(f"diff --git a/{x[0]['old_path']} b/{x[0]['new_path']} {x[0]['diff']}") <= 3000))]

        # Iterate over each row in the filtered DataFrame
        for index, row in lang_df.iterrows():
            diff = row['mods'][0]
            old_path = 'a/' + diff['old_path']
            new_path = 'b/' + diff['new_path']
            diff_content = diff['diff']  # assume diff_content is an empty string
            item = {
                'msg': row['message'],
                'diff': f"diff --git {old_path} {new_path} {diff_content}",
                'lang': lang
            }
            dfs[lang].append(item)
            bars[lang].update(1)

            # Check if the language has reached 100,000 rows
            if len(dfs[lang]) >= 100000:
                print(f"Reached 100,000 rows for {lang}")
                languages.remove(lang)  # Remove language from list to avoid further processing
                break
    # Break out of the loop if all languages have reached 100,000 rows
    if not languages:
        break

Java:   0%|          | 0/100000 [00:00<?, ?it/s]

Python:   0%|          | 0/100000 [00:00<?, ?it/s]

C#:   0%|          | 0/100000 [00:00<?, ?it/s]

C++:   0%|          | 0/100000 [00:00<?, ?it/s]

JavaScript:   0%|          | 0/100000 [00:00<?, ?it/s]

Reached 100,000 rows for Python
Reached 100,000 rows for JavaScript
Reached 100,000 rows for Java
Reached 100,000 rows for C++
Reached 100,000 rows for C#


In [34]:
import json

# Create a single list of all items
all_items = [item for lang in dfs for item in dfs[lang]]

In [40]:
all_items[9999]

{'msg': 'Add progress-bar dependency to bom',
 'diff': "diff --git a/webjars-bom/pom.xml b/webjars-bom/pom.xml <artifactId>vaadin-upload</artifactId>\n<version>2.1.2</version>\n</dependency>\n+ <dependency>\n+ <groupId>org.webjars.bower</groupId>\n+ <artifactId>vaadin-progress-bar</artifactId>\n+ <version>1.0.0-alpha5</version>\n+ </dependency>\n<!-- WebJars' transitive dependencies resolutions -->\n<dependency>\n",
 'lang': 'Java'}

In [36]:

# Dump the data to a JSON file
with open('../data/chronicle/chronicle_rag_db.json', 'w') as f:
    json.dump(all_items, f, indent=4)

In [2]:
import pickle

In [42]:
data = {}
for index, item in enumerate(all_items):
    data[index+1] = item['msg']

with open("../data/chronicle/rag_msg.pkl", "wb") as f:
    pickle.dump(data, f)

In [17]:
with open("../data/chronicle/rag_msg.pkl", "rb") as f:
    msg_data = pickle.load(f)

In [4]:
from langchain_community.document_loaders import JSONLoader

In [ ]:
diff_loader = JSONLoader(
    file_path='../data/chronicle/chronicle_rag_db.json',
    jq_schema='.[].diff',
    text_content=False)

diff_data = diff_loader.load()

In [53]:

language_loader = JSONLoader(
    file_path='../data/chronicle/chronicle_rag_db.json',
    jq_schema='.[].lang',
    text_content=False)

language_data = language_loader.load()

In [55]:
# 1-1000 data use java splitter, 1000-2000 use cpp splitter, 2000-3000 use csharp splitter, 3000-4000 use python splitter, 4000-5000 use javascript splitter
from langchain.text_splitter import Language
from langchain.text_splitter import RecursiveCharacterTextSplitter

languages = [Language.JAVA, Language.CPP, Language.CSHARP, Language.PYTHON, Language.JS]
splitters = [RecursiveCharacterTextSplitter.from_language(language, chunk_size=300, chunk_overlap=0) for language in languages]


language_dict = {'java': 0, 'c++': 1, 'c#': 2, 'python': 3, 'javascript': 4}

In [56]:
from tqdm.auto import tqdm
diff_split = []
for i, doc in tqdm(enumerate(diff_data), total=len(diff_data), desc="Processing documents"):
    diff_split += splitters[language_dict[language_data[i].page_content.lower()]].split_documents([doc])

Processing documents:   0%|          | 0/500000 [00:00<?, ?it/s]

In [57]:
len(diff_split)

2082804

In [58]:
diff_split[0].page_content

'diff --git a/testsuite/integration-arquillian/tests/base/src/test/java/org/keycloak/testsuite/admin/partialimport/PartialImportTest.java b/testsuite/integration-arquillian/tests/base/src/test/java/org/keycloak/testsuite/admin/partialimport/PartialImportTest.java @@ -133,7 +133,7 @@ public class'

In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

In [6]:
# Define the path to the pre-trained model you want to use
modelPath = "mixedbread-ai/mxbai-embed-large-v1"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    cache_folder = '../models',
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

In [8]:
from langchain_community.vectorstores import Chroma

In [9]:
db = Chroma(persist_directory="./chroma_chronicle_db", embedding_function=embeddings)

In [62]:

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
db = Chroma.from_documents(diff_data, embeddings, persist_directory="./chroma_chronicle_db")

In [ ]:
db.persist()

In [10]:
test_diff_loader = JSONLoader(
    file_path='../data/chronicle/chronicle_data.json',
    jq_schema='.[].diff',
    text_content=False)

test_diff_data = test_diff_loader.load()

In [11]:
def similarity_search(documents):
    # Initialize an empty dictionary to store aggregate scores for each candidate ID
    aggregate_scores = {}

    # Iterate through each document in the documents list
    for document in documents:
        # Apply similarity search function to the document
        results = db.similarity_search_with_relevance_scores(document.page_content, score_threshold=0.0)
        # Iterate through the results for each document
        for candidate_doc in results:
            id = candidate_doc[0].metadata['seq_num']
            score = candidate_doc[1]
            # Update the aggregate score for the candidate ID
            aggregate_scores[id] = aggregate_scores.get(id, 0) + score

    # Find the candidate ID with the highest aggregate score
    max_candidate_id = max(aggregate_scores, key=aggregate_scores.get)
    return max_candidate_id

In [12]:
retriever = db.as_retriever()

In [13]:
from tqdm.auto import tqdm

In [20]:
import random 
random.seed(42)
random.shuffle(test_diff_data)

In [21]:
similar_diff = []
for i, test_data in tqdm(enumerate(test_diff_data), total=len(test_diff_data), desc="Processing documents"):
    query = test_data.page_content
    similar_diff.append(retriever.get_relevant_documents(query)[0])

Processing documents:   0%|          | 0/5000 [00:00<?, ?it/s]

In [23]:
import json
data = []

for sim_diff, test_diff in zip(similar_diff, test_diff_data):
    item = {
        'sim_msg': msg_data[sim_diff.metadata['seq_num']],
        'sim_diff': sim_diff.page_content,
        'org_diff': test_diff.page_content
    }
    data.append(item)

In [24]:
# Write the data to a JSON file
with open('../data/chronicle/data_with_similar_diff.json', 'w', encoding='UTF-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
query =test_diff_data[0].page_content
documents = splitters[0].create_documents([query])

In [ ]:
docs = retriever.get_relevant_documents(documents[0].page_content)

In [ ]:
db.similarity_search_with_relevance_scores(documents[0].page_content)[1]

(Document(page_content='diff --git a/MPChartLib/src/main/java/com/github/mikephil/charting/renderer/BubbleChartRenderer.java \nppp b/MPChartLib/src/main/java/com/github/mikephil/charting/renderer/BubbleChartRenderer.java \nprotected void drawDataSet(Canvas c,IBubbleDataSet dataSet){ \nif(! mViewPortHandler.isInBoundsRight(pointBuffer[0]- shapeHalf)) \nbreak;- final int color=dataSet.getColor(( int)entry.getX());+final int color=dataSet.getColor(j); \nmRenderPaint.setColor(color); \nc.drawCircle(pointBuffer[0], pointBuffer[1], shapeHalf,mRenderPaint); \n', metadata={'seq_num': 27766, 'source': 'D:\\TU Delft\\thesis\\LLM_CMG\\llm4commit\\data\\rag_data.json'}),
 0.736459085862412)

Processing documents: 100%|██████████| 5000/5000 [1:46:31<00:00,  1.28s/it]  


In [ ]:
with open('../data/msg_nngen_nmt_codebert_chatgpt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_rag'] = msg

output_file = '../data/msg_nngen_nmt_codebert_chatgpt_rag.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

In [ ]:
len(gpt_msg)

5000